In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [4]:
for k in [1..5]:
    print((k,broadhurst(k)))

(1, 1)
(2, 47)
(3, 2488)
(4, 138799)
(5, 7976456)


In [5]:
order(2,3/4)

-2

In [6]:
print(order(2,1/205915392))

-8


In [4]:
data=[]
for n in [2..99]:
    nsz2=order(2,ZZ(broadhurst(n)))
    ds2=digitsum(2,n)
    data=data+[nsz2-3*ds2+3]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
residues=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    v=evaluate(polyd,3)
    residue=mod(v,2)
    residues=residues+[residue]
print(residues) # so, within the range of observations,ord(2,cst term of j_3^power) = ord(2,nf)
# i.e., ord(2,cst term of j^power) = ord(2,nf) and per cell above, ord(2,nf)
# obeys the 2-order rule in my '98 Exp. Math. article.

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
polydata=[]
for power in [1..10]:
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    print("=================================================================================")
    print("power: ",power)
    print("constant term polynomial:")
    print(rl.factor())
import pickle
wfile = open('/Users/barrybrent/25oct22no1.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

power:  1
constant term polynomial:
(24) * x * (x^2 + 4/3)
power:  2
constant term polynomial:
(1128) * x^2 * (x^4 + 184/141*x^2 + 80/141)
power:  3
constant term polynomial:
(59712) * x^3 * (x^6 + 10868/8397*x^4 + 6128/8397*x^2 + 1472/8397)
power:  4
constant term polynomial:
(3331176) * x^4 * (x^8 + 1609808/1249191*x^6 + 3020000/3747573*x^4 + 1062656/3747573*x^2 + 173312/3747573)
power:  5
constant term polynomial:
(191434944) * x^5 * (x^10 + 288329564/224337825*x^8 + 114391904/134602695*x^6 + 79349632/224337825*x^4 + 4059392/44867565*x^2 + 7488512/673013475)
power:  6
constant term polynomial:
(11213572800) * x^6 * (x^12 + 50576426648/39422716875*x^10 + 311692553296/354804451875*x^8 + 142748677376/354804451875*x^6 + 15011315456/118268150625*x^4 + 9074235392/354804451875*x^2 + 894939136/354804451875)
power:  7
constant term polynomial:
(665672363520) * x^7 * (x^14 + 6121847621476/4778019406125*x^12 + 788619771664/877595401125*x^10 + 8069939288128/18429503423625*x^8 + 20171733511424/1

In [14]:
polydata=[]
numericalfactordata=[]
for power in [1..10]:
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=numericalfactor(rl)
    numericalfactordata=numericalfactordata+[nf/24]
    print("=================================================================================")
    print("power: ",power)
    print("constant term polynomial:")
    print(rl.factor());print()
    print("numerical term:")
    print(factor(ZZ(nf)));print()
import pickle
wfile = open('/Users/barrybrent/25oct22no1.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("numerical factors divided by 24:")
print(numericalfactordata)

power:  1
constant term polynomial:
(24) * x * (x^2 + 4/3)

numerical term:
2^3 * 3

power:  2
constant term polynomial:
(1128) * x^2 * (x^4 + 184/141*x^2 + 80/141)

numerical term:
2^3 * 3 * 47

power:  3
constant term polynomial:
(59712) * x^3 * (x^6 + 10868/8397*x^4 + 6128/8397*x^2 + 1472/8397)

numerical term:
2^6 * 3 * 311

power:  4
constant term polynomial:
(3331176) * x^4 * (x^8 + 1609808/1249191*x^6 + 3020000/3747573*x^4 + 1062656/3747573*x^2 + 173312/3747573)

numerical term:
2^3 * 3 * 138799

power:  5
constant term polynomial:
(191434944) * x^5 * (x^10 + 288329564/224337825*x^8 + 114391904/134602695*x^6 + 79349632/224337825*x^4 + 4059392/44867565*x^2 + 7488512/673013475)

numerical term:
2^6 * 3 * 997057

power:  6
constant term polynomial:
(11213572800) * x^6 * (x^12 + 50576426648/39422716875*x^10 + 311692553296/354804451875*x^8 + 142748677376/354804451875*x^6 + 15011315456/118268150625*x^4 + 9074235392/354804451875*x^2 + 894939136/354804451875)

numerical term:
2^6 * 3 * 

In [ ]:
# For the above numerical factors/24, see OEIS A005148 (henceforth "Newman-Shanks sequence"), 
# which cites Newman-Shanks. In his appendix to 
# Newman-Shanks, Zagier defines f(tau) = 1 + 24 Sum__{n=1}^infty sigma^*(n) q^n with tau & q as usual.
# (Here sigma^* is the sum over odd divisors.)
# sets U(q) = q Prod__{n=1}^infty  (1 + q^n)^24,and shows that the Newman-Shanks a_n   satisfies
# 1\f(tau) = 1 + 24 Sum_{n=1}^infty (-1)^n a_n U^n in a tau-neighborhood of infinity.
# So for given n, I think we can compare our sequence just above here with the {a_1, ..., a_n}.

In [33]:
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
for power in [1..10]:
    broadhurst_values=broadhurst_values+[broadhurst(power)]
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=numericalfactor(rl)
    numericalfactordata=numericalfactordata+[nf/24]
print("numerical factors divided by 24:")
print(numericalfactordata)
print(broadhurst_values)

numerical factors divided by 24:
[1, 47, 2488, 138799, 7976456, 467232200, 27736348480, 1662803271215, 100442427373480, 6103747246289272]
[1, 47, 2488, 138799, 7976456, 467232200, 27736348480, 1662803271215, 100442427373480, 6103747246289272]


In [38]:
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..10]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)

(power,difference,time):  (1, 0, 0.24236702919006348)
(power,difference,time):  (2, 0, 0.47910094261169434)
(power,difference,time):  (3, 0, 0.7627589702606201)
(power,difference,time):  (4, 0, 1.0442099571228027)
(power,difference,time):  (5, 0, 1.3331999778747559)
(power,difference,time):  (6, 0, 1.7196261882781982)
(power,difference,time):  (7, 0, 2.0416369438171387)
(power,difference,time):  (8, 0, 2.375241994857788)
(power,difference,time):  (9, 0, 2.8045480251312256)
(power,difference,time):  (10, 0, 3.173133134841919)
tests:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)

(power,difference,time):  (1, 0, 0.3144361972808838)
(power,difference,time):  (2, 0, 0.5282609462738037)
(power,difference,time):  (3, 0, 0.8055422306060791)
(power,difference,time):  (4, 0, 1.1587200164794922)
(power,difference,time):  (5, 0, 1.5109760761260986)
(power,difference,time):  (6, 0, 1.8811841011047363)
(power,difference,time):  (7, 0, 2.220677137374878)
(power,difference,time):  (8, 0, 2.688606023788452)
(power,difference,time):  (9, 0, 3.0114312171936035)
(power,difference,time):  (10, 0, 3.4133079051971436)
(power,difference,time):  (11, 0, 3.834355115890503)
(power,difference,time):  (12, 0, 4.359604120254517)
(power,difference,time):  (13, 0, 4.761638879776001)
(power,difference,time):  (14, 0, 5.0981810092926025)
(power,difference,time):  (15, 0, 5.740916967391968)
(power,difference,time):  (16, 0, 6.207465171813965)
(power,difference,time):  (17, 0, 6.851208209991455)
(power,difference,time):  (18, 0, 7.197594165802002)
(power,difference,time):  (19, 0, 7.3150620460

KeyboardInterrupt: 

In [3]:
import pickle
wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)


tests:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

(power,difference,time):  (95, 0, 4264.023776769638)
(power,difference,time):  (96, 0, 4331.05419921875)
(power,difference,time):  (97, 0, 4742.513250112534)
(power,difference,time):  (98, 0, 5053.908270120621)
(power,difference,time):  (99, 0, 5381.917376041412)
(power,difference,time):  (100, 0, 5689.437500715256)
tests:
[0, 0, 0, 0, 0, 0]
tests:
[0, 0, 0, 0, 0, 0]


In [11]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^0
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[power-v2]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 

In [9]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    no=[]
    for j in [1..100]: # j = 0 allows m<3
        a=2*j+1
        m=a*2^0
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        if v2!=0:no=no+[(power,m)]  
        po2=order(2,power)
        data=data+[po2-v2]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print("(power,lndata): ",(power,lndata))
    print("(power,no): ",no) # always empty: v2 = 0 always
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(power,lndata):  (1, 1)
(power,no):  []
(power,lndata):  (2, 1)
(power,no):  []
(power,lndata):  (3, 1)
(power,no):  []
(power,lndata):  (4, 1)
(power,no):  []
(power,lndata):  (5, 1)
(power,no):  []
(power,lndata):  (6, 1)
(power,no):  []
(power,lndata):  (7, 1)
(power,no):  []
(power,lndata):  (8, 1)
(power,no):  []
(power,lndata):  (9, 1)
(power,no):  []
(power,lndata):  (10, 1)
(power,no):  []
(power,lndata):  (11, 1)
(power,no):  []
(power,lndata):  (12, 1)
(power,no):  []
(power,lndata):  (13, 1)
(power,no):  []
(power,lndata):  (14, 1)
(power,no):  []
(power,lndata):  (15, 1)
(power,no):  []
(power,lndata):  (16, 1)
(power,no):  []
(power,lndata):  (17, 1)
(power,no):  []
(power,lndata):  (18, 1)
(power,no):  []
(power,lndata):  (19, 1)
(power,no):  []
(power,lndata):  (20, 1)
(power,no):  []
(power,lndata):  (21, 1)
(power,no):  []
(power,lndata):  (22, 1)
(power,no):  []
(power,lndata):  (23, 1)
(power,no):  []
(power,lndata):  (24, 1)
(power,no):  []
(power,lndata):  (25, 1)


In [15]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^1
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[v2-power]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[4, 10, 14, 22, 26, 32, 36, 46, 50, 56, 60, 68, 72, 78, 82, 94, 98, 104, 108, 116, 120, 126, 130, 140, 144, 150, 154, 162, 166, 172, 176, 190, 194, 200, 204, 212, 216, 222, 226, 236, 240, 246, 250, 258, 262, 268, 272, 284, 288, 294, 298, 30

In [17]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^2
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[v2-power]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 

In [18]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^3
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[v2-power]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120, 124, 128, 132, 136, 140, 144, 148, 152, 156, 160, 164, 168, 172, 176, 180, 184, 188, 192, 196, 200, 204, 208, 212, 

In [19]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^4
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[v2-power]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260,

In [20]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^5
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[v2-power]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306, 3

In [21]:
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    for j in [0..100]:
        a=2*j+1
        m=a*2^6
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        data=data+[v2-power]
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print((power,lndata))
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
(25, 1)
(26, 1)
(27, 1)
(28, 1)
(29, 1)
(30, 1)
(31, 1)
(32, 1)
(33, 1)
(34, 1)
(35, 1)
(36, 1)
(37, 1)
(38, 1)
(39, 1)
(40, 1)
(41, 1)
(42, 1)
(43, 1)
(44, 1)
(45, 1)
(46, 1)
(47, 1)
(48, 1)
(49, 1)
(50, 1)
(51, 1)
(52, 1)
(53, 1)
(54, 1)
(55, 1)
(56, 1)
(57, 1)
(58, 1)
(59, 1)
(60, 1)
(61, 1)
(62, 1)
(63, 1)
(64, 1)
(65, 1)
(66, 1)
(67, 1)
(68, 1)
(69, 1)
(70, 1)
(71, 1)
(72, 1)
(73, 1)
(74, 1)
(75, 1)
(76, 1)
(77, 1)
(78, 1)
(79, 1)
(80, 1)
(81, 1)
(82, 1)
(83, 1)
(84, 1)
(85, 1)
(86, 1)
(87, 1)
(88, 1)
(89, 1)
(90, 1)
(91, 1)
(92, 1)
(93, 1)
(94, 1)

initial terms:

[7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 175, 182, 189, 196, 203, 210, 217, 224, 231, 238, 245, 252, 259, 266, 273, 280, 287, 294, 301, 308, 315, 322, 329, 336, 343, 350, 357,

In [19]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=[]
for k in [0..93]:
    data=[]
    power=s[k][0]
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    no=[]
    for j in [1..100]: # j = 0 allows m<3
        a=2*j+1
        m=a*2^1
        polydm=evaluate(polyd,m)
        v2=order(2,polydm)
        #if v2!=0:no=no+[(power,m)]
        po2=order(2,power)
        data=data+[v2/(po2+1)] #avoids division by zero
    sdata=Set(data);ldata=list(sdata);lndata=len(ldata)
    print("(power,lndata): ",(power,lndata)) #lndata=1  => v2 - po2 is constant for given power;
    # i.e., v2 is strictly a function of order(2,power).
    #print("(power,no): ",no)
    ls=ls+[data[0]]
print();print("initial terms:");print();print(ls);print();
print("difference table:");print();print(difference_table(ls))

(power,lndata):  (1, 1)
(power,lndata):  (2, 1)
(power,lndata):  (3, 1)
(power,lndata):  (4, 1)
(power,lndata):  (5, 1)
(power,lndata):  (6, 1)
(power,lndata):  (7, 1)
(power,lndata):  (8, 1)
(power,lndata):  (9, 1)
(power,lndata):  (10, 1)
(power,lndata):  (11, 1)
(power,lndata):  (12, 1)
(power,lndata):  (13, 1)
(power,lndata):  (14, 1)
(power,lndata):  (15, 1)
(power,lndata):  (16, 1)
(power,lndata):  (17, 1)
(power,lndata):  (18, 1)
(power,lndata):  (19, 1)
(power,lndata):  (20, 1)
(power,lndata):  (21, 1)
(power,lndata):  (22, 1)
(power,lndata):  (23, 1)
(power,lndata):  (24, 1)
(power,lndata):  (25, 1)
(power,lndata):  (26, 1)
(power,lndata):  (27, 1)
(power,lndata):  (28, 1)
(power,lndata):  (29, 1)
(power,lndata):  (30, 1)
(power,lndata):  (31, 1)
(power,lndata):  (32, 1)
(power,lndata):  (33, 1)
(power,lndata):  (34, 1)
(power,lndata):  (35, 1)
(power,lndata):  (36, 1)
(power,lndata):  (37, 1)
(power,lndata):  (38, 1)
(power,lndata):  (39, 1)
(power,lndata):  (40, 1)
(power,ln